# MAT281 - Tarea 2

**Indicaciones**:

* En los **Ejercicio 1-8** puedes utilizar tanto `matplotlib` como `altair` según te parezca más conveniente o cómodo, en ambos casos cada gráfico debe tener elementos mínimos como:
    - Título
    - Nombre de los ejes, leyendas, etc. en formato _amigable_/_humano_, por ejemplo, si la columna del dataframe en cuestión tiene por nombre `casos_confirmados` se espera que el eje del gráfico tenga por nombre `Casos confirmados`.
    - Colores adecuados al tipo de datos.
    - Un tamaño adecuado para ver con facilidad en una pantalla con resolución HD o FullHD.
    - Cada vez que no se cumplan alguna de estos requerimientos se descontará __1 punto__ de la nota final.

* Para el **Ejercicio 9** es obligación utilizar `altair`.
* Cada ejercicio debe estar acompañado con una celda con comentarios o análisis que puedas desprender de los gráficos.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import ipywidgets as widgets

from datetime import date
from ipywidgets import interactive, interact

pd.set_option('display.max_columns', 999)
alt.data_transformers.disable_max_rows()
alt.themes.enable('opaque')

%matplotlib inline

**COVID-19 en Chile** 

En esta tarea exploraremos los datos de Covid-19 en Chile a profundidad. Las siguientes celdas cargarán los datos a utilizar en tu sesión. Es importante que leas la documentación de cada conjunto de datos para comprender las columnas.

In [ ]:
start_date = pd.to_datetime("2020-04-13")

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto6
confirmados = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto6/bulk/data.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
    .dropna()
    .astype({"casos_confirmados": np.float, "tasa": np.float})
)

confirmados.head()

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto19
activos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto19/CasosActivosPorComuna.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .loc[lambda x: x["codigo_comuna"].notnull()]
    .melt(id_vars=["region", "codigo_region", "comuna", "codigo_comuna", "poblacion"], var_name="fecha", value_name="casos_activos")
    .assign(fecha=lambda x: pd.to_datetime(x["fecha"]))
    .loc[lambda x: x["fecha"] >= start_date]
)

activos.head()

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto14
fallecidos = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto14/FallecidosCumulativo.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["region"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos.head()

In [ ]:
# https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto10
fallecidos_etareo = (
    pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario.csv")
    .rename(columns=lambda x: x.lower().replace(" ", "_"))
    .melt(id_vars=["grupo_de_edad"], var_name="fecha", value_name="fallecidos")
    .assign(
        fecha=lambda x: pd.to_datetime(x["fecha"]),
        grupo_de_edad=lambda x: x["grupo_de_edad"].str.replace("<=39", "0-39")
    )
    .loc[lambda x: x["fecha"] >= start_date]
)

fallecidos_etareo.tail()

## Ejercicio 1

(10 puntos)

Mostrar cantidad de fallecidos la fecha por cada grupo etáreo.

In [ ]:
fallecidos_etareo_final=fallecidos_etareo.groupby(["grupo_de_edad"]).aggregate("max").reset_index()

plot = alt.Chart(fallecidos_etareo_final).mark_bar().encode(
    y=alt.Y("grupo_de_edad",title="Grupo de edad"),
    x=alt.X("fallecidos:Q",title="Fallecidos"),
    color=alt.Color("grupo_de_edad:N",legend=None,scale=alt.Scale(scheme='darkblue',reverse=True))
).properties(title="Fallecidos por grupo de edad a la fecha",height=400,width=500,)
text = plot.mark_text(
    align='left',
    baseline='middle',
    dx=2,
    dy=-2
).encode(
    text="fallecidos:Q"
)
(plot + text).configure_axis(grid=False)

**Comentarios:** RESPONDE AQUÍ

## Ejercicio 2

(10 puntos)

¿Qué tan variable es la población de las comunas de Chile? Considera utilizar un gráfico que resuma de buena forma la información sin agregar la variable de región o provincia.

In [ ]:
plot = alt.Chart(confirmados).mark_line().encode(
    x=alt.X("fecha",title="Fecha"),
    y=alt.Y("poblacion",title="Población"),
    color=alt.Color("comuna",title="Comunas")
).properties(height=500,title="Población de las comunas en Chile a lo largo del tiempo")

points = alt.Chart(confirmados).mark_circle(size=0).encode(
  color='comuna'
)

plot + points

**Comentarios:** RESPONDE AQUÍ

## Ejercicio 3

(10 puntos)

Mostrar evolución y comparación de los fallecimientos entre distintos grupos etáreos, pero que al mismo tiempo sea fácil identificar la cantidad de fallecidos total en cada fecha.

In [ ]:
dftotal=fallecidos_etareo.groupby(["fecha"]).aggregate("sum").reset_index().assign(grupo_de_edad="total")
fallecidos_etareo_ex=fallecidos_etareo.append([dftotal])

In [ ]:
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['fecha'], empty='none', resolve='global')

line = alt.Chart(fallecidos_etareo).mark_line(interpolate='basis').encode(
    x=alt.X('fecha',title="Fecha"),
    y=alt.Y('fallecidos:Q',title="Fallecidos"),
    color=alt.Color("grupo_de_edad:N",title="Grupo de edad",scale=alt.Scale(scheme='category10'))
)

selectors = alt.Chart(fallecidos_etareo_ex).mark_point().encode(
    x='fecha',
    opacity=alt.value(0),
).add_selection(nearest)

points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)) )

text = line.mark_text(align='right', dx=-2, dy=-5).encode(
    text=alt.condition(nearest, 'fallecidos:Q', alt.value(' ')) )


rules = alt.Chart(fallecidos_etareo_ex).mark_rule(color='gray').encode(
    x='fecha',
).transform_filter(nearest)


totalline=alt.Chart(dftotal).mark_line(interpolate='basis').encode(
    x=alt.X('fecha',title="Fecha"),
    y=alt.Y('fallecidos:Q',title="Fallecidos"),
    color=alt.Color("grupo_de_edad:N",title="Grupo de edad",scale=alt.Scale(scheme='category10')),
    tooltip='fallecidos:Q'
)

totalpoints = totalline.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)) )

totaltext = totalline.mark_text(align='right', dx=-2, dy=-5).encode(
    text=alt.condition(nearest, 'fallecidos:Q', alt.value(' ')) )

datetext = totalline.mark_text(align='left').encode(
    text=alt.condition(nearest, 'fecha:T', alt.value(' ')),
    x=alt.value(5),
    y=alt.value(10)
).transform_calculate(label='datum.fecha + " inches"')

plot=alt.layer(
    line, selectors, points, rules, text, datetext
).properties(title="Fallecidos por grupo etáreo",
    width=400,
    height=400 )

totalplot=alt.layer(
    totalline, selectors, totalpoints, rules, totaltext, datetext
).properties(title="Total fallecidos por grupo etáreo",
    width=400,
    height=400 )

plot | totalplot

**Comentarios:** RESPONDE AQUÍ

## Ejercicio 4

(10 puntos)

Mostrar en tres gráficos la evolución de casos confirmados, evolución de fallecimientos y evolución de casos activos.

In [ ]:
evo_confirmados = alt.Chart(confirmados).mark_line(interpolate='basis').encode(
    x=alt.X("fecha:T",title="Fecha"),
    y=alt.Y("sum(casos_confirmados)",title="Casos confirmados")
).properties(title="Evolución de casos confirmados",height=350,width=300)

evo_fallecidos = alt.Chart(fallecidos).mark_line().encode(
    x=alt.X("fecha:T",title="Fecha"),
    y=alt.Y("sum(fallecidos)",title="Fallecidos")
).properties(title="Evolución de fallecidos",height=350,width=300)

evo_activos = alt.Chart(activos).mark_line().encode(
    x=alt.X("fecha:T",title="Fecha"),
    y=alt.Y("sum(casos_activos)",title="Casos activos")
).properties(title="Evolución de casos activos",height=350,width=300)

evo_confirmados|evo_fallecidos|evo_activos

**Comentarios:** RESPONDE AQUÍ

## Ejercicio 5

(10 puntos)

Comparar la tasa de incidencia entre las regiones a lo largo del tiempo.

In [ ]:
tasaregional = confirmados.drop(columns=["provincia_id","provincia","comuna_id","comuna","tasa"]).groupby(["fecha","region_id","region"]).aggregate("sum").reset_index()

tasaregional["tasa"]=100000*tasaregional["casos_confirmados"]/tasaregional["poblacion"]

In [ ]:
highlight = alt.selection( type='single', on='mouseover', fields=["region"],nearest=True)

base = alt.Chart(tasaregional).encode(
    x=alt.X("fecha",title="Fecha"),
    y=alt.Y("tasa:Q",title="Tasa de incidencia"),
    color=alt.Color("region:N", title="Región", scale=alt.Scale(scheme='tableau20')),
).properties(title="Tasa de insidencia regional",
    width=600,
    height=400
)

points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(highlight).properties()

lines = base.mark_line().encode(
    size=alt.condition(~highlight,alt.value(1),alt.value(3))
)

text = base.mark_text(align='right', dx=-2, dy=-5,interpolate='basis').encode(
    alt.X('fecha:T', aggregate='max',title=""),
    alt.Y('tasa:Q', aggregate={'argmax': 'fecha'},title=""),
    text=alt.condition(highlight, 'region:N', alt.value(' '))
)
points + lines + text

**Comentarios:** RESPONDE AQUÍ

## Ejercicio 6

(10 puntos)

¿Hay alguna conclusión que puedas obtener rápidamente al graficar un _scatter plot_ con los casos confirmados y tasa de incidencia de cada comuna para los días 13 de abril y 6 de noviembre del 2020? Además, colorea cada punto según la región a la que pertenece y considera si es útil en el gráfico que el tamaño sea proporcional a la población.

In [ ]:
date1=pd.to_datetime("2020-04-13")
date2=pd.to_datetime("2020-11-06")

In [ ]:
confirmados1 = confirmados[confirmados["fecha"]==date1]
confirmados2 = confirmados[confirmados["fecha"]==date2]

plot1_tasa = alt.Chart(confirmados1).mark_circle().encode(
    x=alt.X("casos_confirmados:Q",title="Casos confirmados"),
    y=alt.Y("tasa:Q",title="Tasa"),
    color="region",
    size="poblacion",
    tooltip=[alt.Tooltip("region",title="Región"),alt.Tooltip("comuna",title="Comuna")]
).properties(title=f"scatter plot para el día {date1.date()}",height=350)

plot2_tasa = alt.Chart(confirmados2).mark_circle().encode(
    x=alt.X("casos_confirmados:Q",title="Casos confirmados"),
    y=alt.Y("tasa:Q",title="Tasa"),
    color=alt.Color("region",title="Región"),
    size=alt.Size("poblacion",title="Población"),
    tooltip=[alt.Tooltip("region",title="Región"),alt.Tooltip("comuna",title="Comuna")]
).properties(title=f"scatter plot para el día {date2.date()}",height=350)



plot1_tasa | plot2_tasa

**Comentarios:** RESPONDE AQUÍ

## Ejercicio 7

(10 puntos)

1. Grafica la evolución de los casos activos de cada comuna en un solo gráfico. 
2. Grafica la evolución de los casos activos de cada comuna en gráficos separados por región.

Entrega los pros y contras de cada uno de estos enfoques.

In [ ]:
plot = alt.Chart(activos).mark_line().encode(
    x=alt.X("fecha:T",title="Fecha"),
    y=alt.Y("casos_activos:Q",title="Casos activos"),
    color=alt.Color("comuna",title="Comuna"),
    tooltip="comuna"
).properties(height=500,width=700,title="Evolución de casos activos por comuna")
points = alt.Chart(activos).mark_circle(size=0).encode(
  color='comuna'
)
plot + points

In [ ]:
plot = alt.Chart(activos).mark_line().encode(
    x=alt.X("fecha:T",title="Fecha"),
    y=alt.Y("casos_activos:Q",title="Casos activos"),
    color=alt.Color("comuna",title="Comuna"),
    tooltip="comuna"
).properties(title="Evolución de casos activos por comuna (con filtro por Región)",height=500,width=700)

circles = plot.mark_circle().encode(
    x="fecha:T",
    y="casos_activos:Q",
    color=alt.Color("comuna"),
    tooltip="comuna"
).properties()

points = alt.Chart(activos).mark_circle(size=0).encode(
  color='comuna'
)

dropdown = alt.binding_select(options=np.array(activos["region"].unique()))
selector = alt.selection_single(fields=["region"], bind=dropdown, name="s")

plot_selector = plot.add_selection(
    selector
).transform_filter(selector)

circle_selector = circles.add_selection(
    selector
).transform_filter(selector)

plot_selector + points

**Comentarios:** RESPONDE AQUÍ

## Ejercicio 8

(10 puntos)

Hacer un gráfico que permita comparar rápidamente entre regiones su promedio de casos activos , máximo de casos confirmados y fallecidos. Utiliza los valores reales y apoyarlos con colores.

Se adjunta el diccionario `region_names` con tal de reemplazar los nombres de las regiones en los datos `fallecidos` para poder unir con los otros datos. 

In [ ]:
region_names = {
    "Araucanía": "La Araucanía",
    "Aysén": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes": "Magallanes y de la Antártica Chilena",
    "Metropolitana": "Metropolitana de Santiago",
    "O’Higgins": "Libertador General Bernardo O'Higgins",
}
region_names_ = {
    "Araucanía": "La Araucanía",
    "Aysén": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes": "Magallanes y de la Antártica Chilena",
    "Metropolitana": "Metropolitana de Santiago",
    "O’Higgins": "Libertador General Bernardo O'Higgins",
    "Tarapaca": "Tarapacá",
    "Valparaiso": "Valparaíso",
    "Del Libertador General Bernardo O’Higgins": "Libertador General Bernardo O'Higgins",
    "Nuble": "Ñuble",
    "Biobio": "Biobío",
    "La Araucania": "La Araucanía",
    "Los Rios": "Los Ríos",
    "Aysen": "Aysén del General Carlos Ibáñez del Campo",
    "Magallanes y la Antartica": "Magallanes y de la Antártica Chilena"
}

In [ ]:
fallecidos_ = fallecidos[fallecidos["region"]!="Total"].replace(region_names_)

activos_ = activos.replace(region_names_)\
    .drop(columns=["comuna","codigo_comuna","poblacion"])\
    .groupby(["region","codigo_region","fecha"])\
    .aggregate("sum").reset_index()

confirmados_ = confirmados\
    .drop(columns=["provincia_id","provincia","comuna_id","comuna","tasa"])\
    .groupby(["fecha","region_id","region"]).aggregate("sum")\
    .reset_index().rename(columns={"region_id":"codigo_region"}).replace(region_names_)

confirmados_["tasa"]=100000*confirmados_["casos_confirmados"]/confirmados_["poblacion"]

merge = pd.merge(activos_, fallecidos_ , on=["region","fecha"])
merge = pd.merge(merge, confirmados_, on=["region","codigo_region","fecha"])

In [ ]:
selection = alt.selection_interval(bind='scales')

plot = alt.Chart(merge).mark_circle().encode(
    x="max(fallecidos):Q",
    y="max(casos_confirmados):Q",
    size="mean(casos_activos)",
    color="region",
    tooltip=[alt.Tooltip("region",title="Región")]
).properties(height=400,width=700)
plot

**Comentarios:** RESPONDE AQUÍ

## Ejercicio 9


En este ejercicio buscaremos realizar un mini-dashboard respecto al estado de los casos de COVID-19 en Chile, por lo tanto utilizaremos haremos uso de datos geográficos de manera operacional (es decir, no nos preocuparemos de proyecciones en mapas ni nada por el estilo), lo único es que debes instalar `geopandas` en tu ambiente virtual y no olvidar actualizarlo en tu `environment.yml` para luego subirlo a tu repositorio de GitHub.

Con tu ambiente activo (`conda activate mat281`) basta con ejecutar `conda install -c conda-forge geopandas` para instalar `geopandas`.

In [ ]:
import geopandas as gpd
from pathlib import Path

In [ ]:
shp_filepath = Path().resolve().parent / "data" / "regiones_chile.shp"
regiones = gpd.read_file(shp_filepath)
regiones.head()

In [ ]:
type(regiones)

Lo único que tienes que saber es que un `GeoDataFrame` es idéntico a un `DataFrame` salvo que debe poseer una columna llamada `geometry` caracterice los elementros geométricos, que en este casos son polígonos con los límites de las regiones de Chile. 

Para graficar mapas en Altair se debe usar `mark_geoshape`, además, para no preocuparnos de las proyecciones si o si debes declarar lo siguiente que se muestra en la siguiente celda en las propiedades del gráfico. El resto es igual a cualquier otro gráfico de Altair.

In [ ]:
alt.Chart(regiones).mark_geoshape().encode(
).properties(
    projection={'type': 'identity', 'reflectY': True},
    width=250,
    height=600
)

### Ejercicio 9.1

(10 puntos)

Define el `DataFrame` con el nombre `casos_geo` tal que tenga las columnas

* `region`
* `codigo_region`
* `fecha`
* `poblacion`
* `casos_confirmados`
* `tasa`
* `casos_activos`
* `fallecidos`
* `geometry`

Ten mucho cuidado como unes los dataframes `confirmados`, `activos`, `fallecidos` y `regiones`. Idealmente utilizar el código de región, pero en caso que no se encuentren disponibles utilizar el nombre de la región (no olivdar utilizar el diccionario `region_names`).

In [ ]:
regiones_ = regiones.drop(columns=["objectid","cir_sena","area_km","st_area_sh","st_length_"])\
    .rename(columns={"Region":"region","codregion":"codigo_region"})

In [ ]:
casos_geo = pd.merge(merge, regiones_, on=["codigo_region"])\
    .rename(columns={"region_x":"region"})\
    .drop(columns="region_y")

Ejecuta lo siguiente para convertir el DataFrame anterior en un GeoDataFrames

In [ ]:
casos_geo = casos_geo.pipe(lambda x: gpd.GeoDataFrame(x, geometry="geometry"))
casos_geo.head()

### Ejercicio 9.2

(5 puntos)

Modifica la función `covid_chile_chart` tal que reciba una fecha y una columna. Luego, debe filtrar `casos_geo` con registros de la fecha seleccionada y graficar un mapa donde las regiones se colereen según la columna escogida. 

In [ ]:
columnas= {'Población': "poblacion",
'Casos confirmados': "casos_confirmados",
'Tasa': "tasa",
'Casos activos': "casos_activos",
'Fallecidos': "fallecidos"}

In [ ]:
def covid_chile_chart(fecha, col):
    fecha = pd.to_datetime(fecha)
    data = casos_geo[["region","codigo_region","fecha",col,"geometry"]]
    data = data[data["fecha"]==fecha]
    col = col
    chart = alt.Chart(data).mark_geoshape().encode(
        color=alt.Color(col, title=col.capitalize().replace("_", " "), scale=alt.Scale(scheme='lightgreyteal'))
    ).properties(
        projection={'type': 'identity', 'reflectY': True},
        width=150,
        height=400
    )
    
    chart.display()
    return

Prueba con lo siguiente

In [ ]:
fecha = "2020-04-13"
col = "tasa"
covid_chile_chart(fecha, col)

### Ejercicio 9.3

(5 puntos)

Ahora utilizando `widgets` generaremos el dashboard interactivo. Define lo siguiente:

* col_widget: Un `widgets.Dropdown` donde las opciones a seleccionar sean las columnas `poblacion`, `casos_confirmados`, `tasa`, `casos_activos` y `fallecidos`. Además, el argumento `description` debe ser `Columna`.
* fecha_widget: Un `widgets.DatePicker` donde el argumento `description` sea `Fecha`.
* Ambos widgets deben tener el argumento `continuous_update=False`

In [ ]:
import ipywidgets as widgets
from ipywidgets import interactive, interact

In [ ]:
col_widget = widgets.Dropdown(
    options=[('Población', "poblacion"), ('Casos confirmados', "casos_confirmados"), ('Tasa', "tasa"), ('Casos activos', "casos_activos"), ('Fallecidos', "fallecidos")],
    value='casos_activos',
    description='Columna',
    continuous_update=False
)

In [ ]:
start=pd.to_datetime("2020-11-09")
fecha_widget = widgets.DatePicker(
    description='Fecha',
    value=start,
    continuous_update=False
)

Finalmente, haciendo uso de `interactive`, la función `covid_chile_chart` y todos los widgets es posible crear un _dashboard_ interactivo con los datos de Covid-19.

Respira profundo y explora tu creación!

In [ ]:
covid_dashboard = interactive(
    covid_chile_chart,
    fecha=fecha_widget,
    col=col_widget
)
covid_dashboard

**Comentarios:** Ta bonito